## Polytope Climate-DT Feature Extraction Vertical Profile example notebook

This notebook shows how to use earthkit-data and earthkit-plots to pull destination-earth data from LUMI and plot it using earthkit-plots.


Before running the notebook you need to set up your credentials. See the main readme of this repository for different ways to do this or use the following cells to authenticate.

You will need to generate your credentials using the desp-authentication.py script.

This can be run as follows:

In [1]:
%%capture cap
%run ../desp-authentication.py

This will generate a token that can then be used by earthkit and polytope.

In [2]:
output_1 = cap.stdout.split('}\n')
access_token = output_1[-1][0:-1]

# Requirements
To run this notebook install the following:
* pip install earthkit-data
* pip install earthkit-plots
* pip install cf-units         (Optional for unit conversion in maps)

In [1]:
import earthkit.data
import earthkit.regrid
from earthkit.plots.interactive import Chart
from polytope.api import Client

In [4]:
client = Client(
    address="polytope.lumi.apps.dte.destination-earth.eu")


request = {
        "activity": "scenariomip",
        "class": "d1",
        "dataset": "climate-dt",
        "experiment": "ssp3-7.0",
        "generation": "1",
        "levtype": "pl",
        "date": "20210101",
        "model": "ifs-nemo",
        "expver": "0001",
        "param": "131",
        "realization": "1",
        "resolution": "high",
        "stream": "clte",
        "type": "fc",
        "time": "0000",
        "levelist": "1/to/1000",
        "feature": {
            "type": "verticalprofile",
            "points": [[38.9, -9.1]],
        },
    }

file = client.retrieve("destination-earth", request, "data/climate-dt-earthkit-fe-verticalprofile.grib") 

In [2]:
data = earthkit.data.from_source("file", "data/climate-dt-earthkit-fe-verticalprofile.grib") 

In [3]:
data._json()

{'type': 'CoverageCollection',
 'domainType': 'VerticalProfile',
 'coverages': [{'mars:metadata': {'activity': 'scenariomip',
    'class': 'd1',
    'dataset': 'climate-dt',
    'Forecast date': '2021-01-01T00:00:00Z',
    'experiment': 'ssp3-7.0',
    'expver': '0001',
    'generation': '1',
    'levelist': 1,
    'levtype': 'pl',
    'model': 'ifs-nemo',
    'realization': '1',
    'resolution': 'high',
    'stream': 'clte',
    'type': 'fc',
    'number': 0,
    'step': 0},
   'type': 'Coverage',
   'domain': {'type': 'Domain',
    'axes': {'latitude': {'values': [38.873582658567]},
     'longitude': {'values': [350.9033203125]},
     'levelist': {'values': [1,
       5,
       10,
       20,
       30,
       50,
       70,
       100,
       150,
       200,
       250,
       300,
       400,
       500,
       600,
       700,
       850,
       925,
       1000]},
     't': {'values': ['2021-01-01T00:00:00Z']}}},
   'ranges': {'u': {'type': 'NdArray',
     'dataType': 'float',


In [4]:
# Convert data to xarray
da = data.to_xarray()

In [5]:
da

<xarray.Dataset> Size: 416B
Dimensions:    (latitude: 1, longitude: 1, number: 1, datetime: 1, time: 1,
                levelist: 19)
Coordinates:
  * latitude   (latitude) float64 8B 38.87
  * longitude  (longitude) float64 8B 350.9
  * number     (number) int64 8B 0
  * datetime   (datetime) <U20 80B '2021-01-01T00:00:00Z'
  * time       (time) int64 8B 0
  * levelist   (levelist) int64 152B 1 5 10 20 30 50 ... 600 700 850 925 1000
Data variables:
    u          (latitude, longitude, number, datetime, time, levelist) float64 152B ...
Attributes: (12/15)
    activity:       scenariomip
    class:          d1
    dataset:        climate-dt
    Forecast date:  2021-01-01T00:00:00Z
    experiment:     ssp3-7.0
    expver:         0001
    ...             ...
    model:          ifs-nemo
    realization:    1
    resolution:     high
    stream:         clte
    type:           fc
    number:         0

In [10]:
chart = Chart()
fig = chart.fig
fig['layout']['yaxis']['autorange'] = "reversed"
chart.line(da, y="levelist")
chart.fig.update_layout(yaxis={"title": "hPa"})
chart.title("Vertical profile over Lisbon from the DestinE's Climate DT")
chart.show() # Replace with chart.show() in an interactive session!